In [ ]:
import pandas as pd
import glob
from geopandas import GeoDataFrame
import geopandas as gpd
from math import sin, cos, sqrt, atan2, radians
import numpy as np
import matplotlib.pyplot as plt
import dateutil 
import datetime as dt
from datetime import datetime, timedelta
%matplotlib inline
plt.style.use('ggplot')

# Pulling out file names

In [ ]:
file_names = []
for name in glob.glob('d/google_distance_matrix_data/*'):
    name = name.split('/')[-1]
    file_names.append(name)

In [ ]:
file_names

# Creating one DF, and clean out columns I don't need

In [ ]:
df = pd.read_csv('d/google_distance_matrix_data/-499.csv')

In [ ]:
for file in file_names[1:]:
    df_new = pd.read_csv('d/google_distance_matrix_data/'+ file)
    frames = [df, df_new]
    df = pd.concat(frames)

In [ ]:
df = df.reset_index()

In [ ]:
del df['Unnamed: 0']
del df['Timestamp index.1']
del df['index']
del df['Timestamp index']
del df['Timestamp']

In [ ]:
df.info()

# What am I looking for?

0. Time Frame
1. The distance, time and usage count of the top three bikes?
2. Average time, distance and usage count?
3. Total number of km, time, usages?
4. What time of day, day of week do most people use the bike?
5. Estimation of earnings?

#### 1. Time Frame

In [ ]:
def pmam(x):
    x = str(x)
    #x = (':'.join(a+b for a,b in zip(x[::2], x[1::2])))
    if x == 'NaN':
        pass
    try:
        x = str(x[:2] + ':' + x[2:])
        date = dateutil.parser.parse(x)
        return str(date.strftime('%Y.%m.%d. %H:%M:%S'))
    except:
        return 'NaN'

In [ ]:
df['Timestamp index'] = df['Timestamp +2'].apply(pmam)
df['Timestamp index'] = df['Timestamp index'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%Y.%m.%d. %H:%M:%S'))

In [ ]:
print('Starting date:', list(df['Timestamp index'].sort_values())[0])

In [ ]:
print('End date', list(df['Timestamp index'].sort_values())[-1])

#### 2. The distance, time and usage count of the top three bikes?

In [ ]:
df.groupby('id')['Metres'].sum().sort_values(ascending=False).head(3)

In [ ]:
df.groupby('id')['Metres'].sum().sort_values(ascending=False).plot(kind='bar').axes.get_xaxis().set_visible(False)

In [ ]:
for elem in list(df.groupby('id')['Seconds'].sum().sort_values(ascending=False).head(3)):
    m, s = divmod(elem, 60)
    h, m = divmod(m, 60)
    print(h, 'h', m, 'min', s, 'sec')

In [ ]:
df['id'].value_counts(ascending=False).head(3)

#### 3. Average time, distance and usage count?

In [ ]:
m, s = divmod(int(df['Seconds'].mean()), 60) 
print('Median time bike is in use is:', m, 'min', s, 'sec.')

In [ ]:
m = int(df['Metres'].mean()) 
print('Average distance of a bike:', m, 'm')

In [ ]:
u = str(round(df['id'].value_counts().mean() / 14,1))
print('Daily amount of times a bik is used:', u)

#### 4. What time of day, day of week do most people use the bike?

In [ ]:
df.index = df['Timestamp index']

In [ ]:
df.groupby(by=df.index.hour)['id'].count().plot(kind='bar', figsize=(12,6))

In [ ]:
df.groupby(by=df.index.weekday)['id'].count().plot(kind='bar', figsize=(12,6))

#### 6. Estimation of earnings?

In [ ]:
print('Considering every half our started cost 1.50 CHF, we multplying count of lines with that amount:', int(len(df.index)) * 1.5)